# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [152]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [153]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [154]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [155]:
Group_by = data.groupby(['CustomerID','ProductName'])[['Quantity']]\
                       .agg(sum_Quantity=('Quantity', 'sum')).sort_values(by='CustomerID', ascending=False)

In [156]:
Group_by

sum_Quantity
CustomerID ProductName                                
98200      Yogurt - French Vanilla                  25
           Fondant - Icing                          25
           Cookie Dough - Double                    25
           Cornflakes                               25
           Dried Figs                               50
...                                                ...
33         Potatoes - Idaho 100 Count                1
           Pork - Hock And Feet Attached             1
           Pork - Bacon, Double Smoked               1
           Phyllo Dough                              1
           Apricots - Dried                          1

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [157]:
pivotTable = pd.pivot_table(data,index= "ProductName", columns= "CustomerID",values='Quantity',fill_value=0,aggfunc='sum')
pivotTable

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [158]:
euclid_dist_norm = pd.DataFrame(1/(1 + squareform(pdist(pivotTable.T, 'euclidean'))),index=pivotTable.T.index, columns=pivotTable.T.index)

euclid_dist_norm


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [162]:
cosine_simil_norm  = euclid_dist_norm[412].sort_values(ascending=False)[1:6]
cosine_simil_norm


CustomerID
3317    0.085638
3535    0.084959
1008    0.083333
1072    0.082709
639     0.082403
Name: 412, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [174]:
best_records  = Group_by.loc[cosine_simil_norm.index]
best_records 

sum_Quantity
CustomerID ProductName                                  
3317       Nantucket - Pomegranate Pear                1
           Garbage Bags - Clear                        1
           Kiwi                                        1
           Lamb - Ground                               1
           Muffin Batt - Blueberry Passion             1
...                                                  ...
639        Pepper - Paprika, Hungarian                 1
           Pernod                                      1
           Pomello                                     1
           Pork - Hock And Feet Attached               1
           Pork - Loin, Center Cut                     1

[272 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [175]:
aggregate_product = best_records.groupby('ProductName').agg(sum_Quantity=('sum_Quantity', 'sum')).sort_values(by='sum_Quantity', ascending=False)
aggregate_product

,sum_Quantity
ProductName,
Olive - Spread Tapenade,3
Sprouts - Baby Pea Tendrils,3
Wine - Blue Nun Qualitatswein,3
"Veal - Inside, Choice",3
"Pepper - Black, Whole",3
...,...
Flour - Whole Wheat,1
Foam Cup 6 Oz,1
Garlic,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [318]:
merge_table = pivotTable.merge(aggregate_product, left_on='ProductName', right_on='ProductName')
merge_table = merge_table[merge_table[412]==0].reset_index()
recommended = merge_table.sort_values(by= 'sum_Quantity', ascending = False)[['ProductName',412,'sum_Quantity']].head(5)
recommended

,ProductName,412,sum_Quantity
124,"Sauce - Gravy, Au Jus, Mix",0,3
141,Sprouts - Baby Pea Tendrils,0,3
98,Olive - Spread Tapenade,0,3
163,Wine - Blue Nun Qualitatswein,0,3
31,Cheese - Cambozola,0,3


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [343]:
def product_recommendations(Costumer):
    cosine_simil_norm  = euclid_dist_norm[i].sort_values(ascending=False)[1:6]
    # step 5
    cosine_simil_norm = Group_by.loc[cosine_simil_norm.index]
    # step 6
    aggregate_product = best_records.groupby('ProductName').agg(sum_Quantity=('sum_Quantity', 'sum')).sort_values(by='sum_Quantity', ascending=False)
    # step 7
    merge_table = pivotTable.merge(aggregate_product, left_on='ProductName', right_on='ProductName')
    merge_table = merge_table[merge_table[i]==0]
    merge_table = merge_table.reset_index()
    recommended = merge_table.sort_values(by= 'sum_Quantity', ascending = False)[['ProductName','sum_Quantity']].head(5)
    return recommended

In [344]:
recommendations = {}
CustomerIDs= data['CustomerID'].unique().tolist() 

In [345]:
for i in CustomerIDs:
    recommendations[i] = product_recommendations(i)

recommendations

{61288:                        ProductName  sum_Quantity
 113        Olive - Spread Tapenade             3
 185  Wine - Blue Nun Qualitatswein             3
 154   Soup - Campbells Bean Medley             3
 160    Sprouts - Baby Pea Tendrils             3
 32              Cheese - Cambozola             3,
 77352:                      ProductName  sum_Quantity
 125   Sauce - Gravy, Au Jus, Mix             3
 112        Pepper - Black, Whole             3
 29            Cheese - Cambozola             3
 102      Olive - Spread Tapenade             3
 143  Sprouts - Baby Pea Tendrils             3,
 40094:                       ProductName  sum_Quantity
 103       Olive - Spread Tapenade             3
 146   Sprouts - Baby Pea Tendrils             3
 140  Soup - Campbells Bean Medley             3
 159         Veal - Inside, Choice             3
 126    Sauce - Gravy, Au Jus, Mix             3,
 23548:                        ProductName  sum_Quantity
 152    Sprouts - Baby Pea Tendrils  

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [346]:
df_customer_recomendations2 = pd.DataFrame.from_dict(recommendations,
                                                   orient='index').reset_index()

ValueError: Must pass 2-d input. shape=(1000, 5, 2)

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [347]:
dist_calculation = pdist(X=pivotTable, metric='cityblock')
dist_distribution = squareform(dist_calculation)
distance_eucl = pd.DataFrame(1/(1 + squareform(pdist(pivotTable, 'cityblock'))),
                         index=pivotTable.index, columns=pivotTable.index)
distance_eucl.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,1.000000,0.000265,0.000293,0.000323,0.000311,0.000271,0.000276,0.000288,0.000311,0.000320,...,0.000292,0.000287,0.000300,0.000311,0.000297,0.000341,0.000291,0.000295,0.000284,0.000278
"Appetizer - Mini Egg Roll, Shrimp",0.000265,1.000000,0.000279,0.000290,0.000284,0.000248,0.000256,0.000300,0.000292,0.000259,...,0.000274,0.000263,0.000271,0.000285,0.000281,0.000281,0.000273,0.000287,0.000263,0.000269
Appetizer - Mushroom Tart,0.000293,0.000279,1.000000,0.000322,0.000278,0.000278,0.000279,0.000298,0.000308,0.000269,...,0.000288,0.000271,0.000277,0.000342,0.000325,0.000324,0.000297,0.000290,0.000292,0.000282
Appetizer - Sausage Rolls,0.000323,0.000290,0.000322,1.000000,0.000314,0.000312,0.000284,0.000320,0.000299,0.000303,...,0.000308,0.000299,0.000305,0.000365,0.000326,0.000314,0.000303,0.000302,0.000286,0.000290
Apricots - Dried,0.000311,0.000284,0.000278,0.000314,1.000000,0.000279,0.000267,0.000297,0.000288,0.000282,...,0.000281,0.000270,0.000289,0.000324,0.000290,0.000284,0.000295,0.000284,0.000279,0.000285
